 #         <h1 align="center">   **OpenVaccine || EDA || Feature Engineering**</h1>
 
![](http://https://daslab.stanford.edu/site_data/news_img/openvaccine_lores.png)

# **Importing Libraries**

In [ ]:
!pip install forgi

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import forgi.graph.bulge_graph as fgb
import forgi.visual.mplotlib as fvm

import plotly.express as px
from collections import Counter as count
from collections import Counter

# **Reading Data**

In [ ]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines = True).drop('index' , axis = 1)
test = pd.read_json('../input/stanford-covid-vaccine/test.json' , lines = True).drop('index' , axis = 1)
sub = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.T

In [ ]:
test.head()

In [ ]:
sub.head()

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
print("Train data shape: ",train.shape)
print("Test data shape: ",test.shape)
print("Sample submission shape: ",sub.shape)

# **Starting EDA**

In [ ]:
fig, ax = plt.subplots(1 , 2 , figsize = (20 , 5))
train['signal_to_noise'].plot.kde(ax = ax[0])
ax[0].set_title('Signal/Noise')

sns.countplot(data=train,y='SN_filter',ax=ax[1])
ax[1].set_title('SN_filter')

plt.show()

In [ ]:
plt.figure(figsize = (20 , 2))
sns.boxplot(data = train , x = 'signal_to_noise')

plt.show()

In [ ]:
print("Number of samples with -ev signal/noise values: " , train[train['signal_to_noise']<0].shape[0])

Q1 = np.percentile(train['signal_to_noise'] , q = 25)
Q3 = np.percentile(train['signal_to_noise'] , q = 75)
IQR = Q3 - Q1

print("Number of samples with too high signal/noise values", train[train['signal_to_noise'] > Q3 + 1.5 * IQR].shape[0])

In [ ]:
train.seq_length.value_counts()

In [ ]:
test.seq_length.value_counts()

In [ ]:
fig = px.histogram(
    train, 
    "signal_to_noise", 
    nbins=25, 
    title='signal_to_noise column distribution', 
    width=800,
    height=500
)
fig.show()

In [ ]:
ds = train['SN_filter'].value_counts().reset_index()
ds.columns = ['SN_filter', 'count']
fig = px.pie(
    ds, 
    values='count', 
    names="SN_filter", 
    title='SN_filter bar chart', 
    width=500, 
    height=500
)
fig.show()


In [ ]:
plt.figure(figsize = (12 , 5))
n, bins, patches = plt.hist(x = train['signal_to_noise'] , bins = 'auto' , color = '#FFA500' , alpha = 1 , rwidth = 0.80)
plt.grid(axis='y', alpha = 0.75)
plt.xlabel('signal_to_noise')
plt.ylabel('Frequency')
plt.title('signal_to_noise Histogram')
plt.text(12, 110, f"(SN_filter == 1)  : {train['SN_filter'].value_counts()[0]}", fontsize=15)
plt.text(12, 85, f"(SN_filter == 0) : {train['SN_filter'].value_counts()[1]}", fontsize=15)
plt.show()

In [ ]:
def character_count(row):
    _dictionary = {'G': 0,'A': 0, 'C': 0, 'U': 0, '.': 0, '(': 0, ')': 0, 'E': 0, 'S': 0, 'H': 0, 'B': 0, 'X': 0,'I': 0,'M':0}
    _dictionary = {**_dictionary, **dict(Counter(row['sequence']))}
    _dictionary = {**_dictionary, **dict(Counter(row['structure']))}
    _dictionary = {**_dictionary, **dict(Counter(row['predicted_loop_type']))}
    return list(_dictionary.values())

In [ ]:
feature_columns = ['G','A', 'C', 'U', '.', '(', ')', 'E', 'S', 'H', 'B', 'X','I','M']
train[feature_columns] = train.apply(character_count , axis = 1 , result_type = "expand")

In [ ]:
fig, _ax = plt.subplots(nrows = 4 , ncols = 4 , figsize = (20 , 20))
fig.suptitle("Train Data New Features Histograms", fontsize = 20,)

for i,_ax in enumerate(_ax.ravel()[:14]):
    mean_value = train[feature_columns[i]].mean()
    max_value_index , max_value = Counter(train[feature_columns[i]]).most_common(1)[0]
    
    _ax.hist(x=train[feature_columns[i]] , bins = 'auto' , color = '#800000', alpha = 1 , rwidth = 1)
    _ax.set(ylabel = f"'{feature_columns[i]}' Frequency", title= f"'{feature_columns[i]}' Histogram")
    _ax.axvline(x = mean_value , color='#808000' , label= 'Average',linewidth=2)
    _ax.axvline(x = max_value_index , color = 'y' , label = 'Max', linewidth = 2)
    _ax.legend([f"Average : {mean_value:0.2f}" , f"Max Frequency : {max_value}", "Hist"] , loc = "upper right")
    
plt.show()

In [ ]:
# Train Data New Features correlation

corr = train[feature_columns].corr()
sns.set_context("notebook" , font_scale = 1.0 , rc = {"lines.linewidth": 2.5})
plt.figure(figsize=(13,7))
plt.title("Train Data New Features correlation : ")

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 1)] = True
a = sns.heatmap(corr , mask = mask , annot = True , fmt = '.2f' , cmap = "YlGnBu")
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
plt.figure(figsize = (20 , 10))
corr = train.corr()
sns.heatmap(corr , annot = True , fmt = '.2f' , cmap = "PiYG")

In [ ]:
## Test-Data

feature_columns = ['G','A', 'C', 'U', '.', '(', ')', 'E', 'S', 'H', 'B', 'X','I','M']
test[feature_columns] = test.apply(character_count,axis=1,result_type="expand")
fig, _ax = plt.subplots(nrows=4,ncols=4,figsize=(20,20))
fig.suptitle("Test Data New Features Histograms", fontsize=20,)
for i,_ax in enumerate(_ax.ravel()[:14]):
    mean_value = test[feature_columns[i]].mean()
    max_value_index,max_value = Counter(test[feature_columns[i]]).most_common(1)[0]
    
    _ax.hist(x=test[feature_columns[i]],bins='auto', color='#808000', alpha=1, rwidth=1)
    _ax.set(ylabel=f"'{feature_columns[i]}' Frequency", title= f"'{feature_columns[i]}' Histogram")
    _ax.axvline(x=mean_value, color='r', label= 'Average',linewidth=2)
    _ax.axvline(x=max_value_index, color='y', label= 'Max',linewidth=2)
    _ax.legend([f"Average : {mean_value:0.2f}",f"Max Frequency : {max_value}", "Hist"], loc ="upper right")
plt.show()

In [ ]:
# Test Data New Features correlation
corr = test[feature_columns].corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(13,7))
plt.title("Test Data New Features correlation : ")

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 1)] = True
a = sns.heatmap(corr,mask=mask, annot=True, fmt='.2f' , cmap = 'hot')

rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
avg_reactivity = np.array(list(map(np.array,train.reactivity))).mean(axis = 0)
avg_deg_50C = np.array(list(map(np.array,train.deg_50C))).mean(axis = 0)
avg_deg_pH10 = np.array(list(map(np.array,train.deg_pH10))).mean(axis = 0)
avg_deg_Mg_50C = np.array(list(map(np.array,train.deg_Mg_50C))).mean(axis = 0)
avg_deg_Mg_pH10 = np.array(list(map(np.array,train.deg_Mg_pH10))).mean(axis = 0)

In [ ]:
fig , ax = plt.subplots(1 , 3 , figsize = (20 , 4))

sns.kdeplot(avg_reactivity , ax = ax[0])
ax[0].set_title("Distribution of Reactivity average over position" , size = 15)

# Distribution of deg_50C Averaged over position
sns.kdeplot(avg_deg_50C , ax = ax[1])
ax[1].set_title('Distribution of deg_50C Averaged over position', size = 15)

# Distribution of deg_pH10 Averaged over position
sns.kdeplot(avg_deg_pH10 , ax = ax[2])
ax[2].set_title('Distribution of deg_pH10 Averaged over position' , size = 15)


plt.show()

In [ ]:
plt.figure(figsize = (20 , 8))

sns.lineplot(x = range(68) , y = avg_reactivity , label = 'reactivity')
sns.lineplot(x = range(68) , y = avg_deg_50C , label = 'deg_50C')
sns.lineplot(x = range(68) , y = avg_deg_pH10 , label = 'deg_ph10')

plt.xlabel('Positions')
plt.xticks(range(0 , 68 , 2))
plt.ylabel('Values')
plt.title('Average Target Values (w/o Mg) V/S Positions')

plt.show()

We can see high values of Degradation & Reactivity at the beginning of the sequence There is high degradation because of pH10 at the beginning but no such pattern for rest of the positions. We can see some correlation between deg_50C & deg_pH10

In [ ]:
plt.figure(figsize = (20 , 8))
sns.regplot(x = avg_deg_50C , y = avg_deg_pH10)

plt.title('Average deg_50C V/S deg_pH10')
plt.ylabel('deg_50C')
plt.xlabel('deg_pH10')

plt.show()

In [ ]:
print("Correlation Coeff between avg_deg_50C & avg_deg_pH10: ",np.corrcoef(avg_deg_50C,avg_deg_pH10)[0][1])

In [ ]:
plt.figure(figsize = (20 , 8))
sns.regplot(x = avg_deg_50C , y = avg_deg_Mg_pH10)

plt.title('Average deg_50C V/S deg_Mg_pH10')
plt.ylabel('deg_50C')
plt.xlabel('deg_pH10')

plt.show()

In [ ]:
plt.figure(figsize = (20 , 8))
sns.regplot(x = avg_deg_50C , y = avg_reactivity)

plt.title('Average deg_50C V/S deg_pH10')
plt.ylabel('deg_50C')
plt.xlabel('deg_pH10')

plt.show()

In [ ]:
fig, ax = plt.subplots(1 , 2 , figsize = (20 , 4))

# Distribution of deg_50C Averaged over position
sns.kdeplot(avg_deg_Mg_50C , ax = ax[0])
ax[0].set_title('Distribution of deg_Mg_50C Averaged over position' , size = 15)


# Distribution of deg_pH10 Averaged over position
sns.kdeplot(avg_deg_Mg_pH10 , ax = ax[1])
ax[1].set_title('Distribution of deg_Mg_pH10 Averaged over position' , size = 15)

plt.show()

In [ ]:
plt.figure(figsize = (20 , 8))

sns.lineplot(x = range(68) , y = avg_deg_Mg_50C , label = 'deg_Mg_50C')
sns.lineplot(x = range(68) , y = avg_deg_Mg_pH10 , label = 'deg_Mg_pH10')

plt.xlabel('Positions')
plt.xticks(range(0 , 68 , 2))
plt.ylabel('Values')
plt.title('Average Target Values (w Mg) V/S Positions')

plt.show()

In [ ]:
print("Correlation Coeff between avg_deg_Mg_50C & avg_deg_Mg_pH10: ",np.corrcoef(avg_deg_Mg_50C,avg_deg_Mg_pH10)[0][1])

In [ ]:
pos = np.random.choice(68)

fig, ax = plt.subplots(1,3,figsize=(20,4))

# Distribution of Reactivity at Random position
sns.kdeplot(np.array(list(map(np.array,train.reactivity)))[:,pos],ax=ax[0])
ax[0].set_title(f'Distribution of Reactivity at position-{pos}',size=15)

# Distribution of deg_50C at Random position
sns.kdeplot(np.array(list(map(np.array,train.deg_50C)))[:,pos],ax=ax[1])
ax[1].set_title(f'Distribution of deg_50C at position-{pos}',size=15)

# Distribution of deg_pH10 at Random position
sns.kdeplot(np.array(list(map(np.array,train.deg_pH10)))[:,pos],ax=ax[2])
ax[2].set_title(f'Distribution of deg_pH10 at position-{pos}',size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,4))

# Distribution of deg_50C at Random position
sns.kdeplot(np.array(list(map(np.array,train.deg_Mg_50C)))[:,pos],ax=ax[0])
ax[0].set_title(f'Distribution of deg_Mg_50C at position-{pos}',size=15)


# Distribution of deg_pH10 at Random position
sns.kdeplot(np.array(list(map(np.array,train.deg_Mg_pH10)))[:,pos],ax=ax[1])
ax[1].set_title(f'Distribution of deg_Mg_pH10 at position-{pos}',size=15)

plt.show()

In [ ]:
y = ['reactivity_error' , 'deg_error_Mg_pH10' , 'deg_error_pH10' , 'deg_error_Mg_50C' ,'deg_error_50C']
x = [np.array(list(map(np.array,train[col]))).mean(axis = 0) for col in y]

plt.figure(figsize = (20 , 5))

sns.boxplot(y = y , x = x)

plt.xlabel('Error values')
plt.title('Average Errors in Calculation of Targets')

plt.show()


Very high values of Errors in deg_error_pH10 & deg_error_50C but maybe we should not worry about these targets because evaluation is not based on these targets Maybe this could be a reason why our models will not be scored on these two targets

In [ ]:
plt.figure(figsize = (20 , 8))

for i in range(len(y)):
    sns.lineplot(x = range(68) , y = x[i] , label = y[i])
    
plt.xlabel('Positions')
plt.xticks(range(0,68,2))
plt.ylabel('Error')
plt.title('Error V/S Position')

plt.show()


We can see an interesting pattern here. We have high values of errors between postions 0 and 4, but then it starts decreasing constantly as we go ahead Now let's analyse if all samples are contributing to these high values or there are only few smaples which are driving these values

In [ ]:
pos = np.random.choice(68)

y = ['reactivity_error','deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C','deg_error_50C']
x = [np.array(list(map(np.array , train[col])))[:,pos] for col in y]

plt.figure(figsize = (20 , 5))
plt.title(f'Error Distribution at position - {pos}')
plt.xlabel('Error')

sns.boxplot(y=y,x=x)

plt.show()

We can see there are few samples which are having high values of errors Now let's try to filter these samples We can try using SN_filter for this

In [ ]:
y = ['reactivity_error','deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C','deg_error_50C']
x = [np.array(list(map(np.array,train[train.SN_filter == 1][col])))[:,pos] for col in y]

plt.figure(figsize = (20 , 5))
plt.title(f'(Filtered) Error Distribution at position - {pos}')
plt.xlabel('Error')

sns.boxplot(y = y , x = x)

plt.show()

In [ ]:
def plot_sample(sample):
    
    struct = sample['structure']
    seq = sample['sequence']
    bg = fgb.BulgeGraph.from_fasta_text(f'>rna1\n{struct}\n{seq}')[0]
    
    plt.figure(figsize=(20,8))
    fvm.plot_rna(bg)
    plt.title(f"RNA Structure (id: {sample.id})")
    plt.show()

In [ ]:
sample = train.iloc[np.random.choice(train.shape[0])]
#plot_sample(sample)
print("Predicted Loop type: ",sample['predicted_loop_type'])

In [ ]:
from collections import defaultdict

reactivity = defaultdict(lambda: [])
deg_Mg_50C = defaultdict(lambda: [])
deg_Mg_pH10 = defaultdict(lambda: [])

for i in range(len(sample['reactivity'])):
    reactivity[sample['structure'][i]].append(float(sample['reactivity'][i]))
    deg_Mg_50C[sample['structure'][i]].append(float(sample['deg_Mg_50C'][i]))
    deg_Mg_pH10[sample['structure'][i]].append(float(sample['deg_Mg_pH10'][i]))

plt.figure(figsize = (18 , 5))
for key in reactivity.keys():
    sns.kdeplot(data=reactivity[key],label=key)

plt.title('Structure wise Distribution of Reactivity of a Random Sample')
plt.show()

In [ ]:
plt.figure(figsize = (20 , 5))
for key in reactivity.keys():
    sns.kdeplot(data = deg_Mg_50C[key] , label = key)

plt.title('Structure wise Distribution of deg_Mg_50C of a Random Sample')
plt.show()

In [ ]:

plt.figure(figsize = (20 , 5))
for key in reactivity.keys():
    sns.kdeplot(data = deg_Mg_pH10[key] , label = key)

plt.title('Structure wise Distribution of deg_Mg_pH10 of a Random Sample')
plt.show()

In [ ]:
reactivityDict = defaultdict(lambda: [])

for index in range(train.shape[0]):
    
    sample = train.iloc[index]

    structure = sample['structure']
    sequence = sample['sequence']
    reactivity = sample['reactivity']

    q = []

    for i,s in enumerate(structure[:len(reactivity)]):
        if s=='.':
            reactivityDict[sequence[i]].append(reactivity[i])
        elif s=='(':
            q.append(i)
        elif s==')':
            j = q.pop(0)
            key = "-".join(sorted([sequence[i],sequence[j]]))
            reactivityDict[key].append(reactivity[i])
            reactivityDict[key].append(reactivity[j])

In [ ]:
fig, ax = plt.subplots(len(reactivityDict.keys()) , 1 , figsize = (20 , 2 * len(reactivityDict.keys())) , sharex = True)

for i, key in enumerate(reactivityDict.keys()):
    sns.boxplot(x=reactivityDict[key],ax=ax[i])
    ax[i].set_ylabel(key)

plt.xlabel('Reactivity')
plt.show()

In [ ]:
pri = test[test.seq_length == 130]
pub = test[test.seq_length == 107]

In [ ]:
def read_bpps_mean(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").mean(axis=1))
    return bpps_arr

train['bpps_mean'] = read_bpps_mean(train)
pri['bpps_mean'] = read_bpps_mean(pri)
pub['bpps_mean'] = read_bpps_mean(pub)

sns.distplot(np.array(train['bpps_mean'].to_list()).reshape(-1) , color = "Blue")
sns.distplot(np.array(pub['bpps_mean'].to_list()).reshape(-1) , color = "Green")
sns.distplot(np.array(pri['bpps_mean'].to_list()).reshape(-1) , color = "Red")

In [ ]:

def read_bpps_max(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").max(axis=1))
    return bpps_arr

train['bpps_max'] = read_bpps_max(train)
pri['bpps_max'] = read_bpps_max(pri)
pub['bpps_max'] = read_bpps_max(pub)

sns.distplot(np.array(train['bpps_max'].to_list()).reshape(-1),color = "#FF0000")
sns.distplot(np.array(pub['bpps_max'].to_list()).reshape(-1),color = "#00FFFF")
sns.distplot(np.array(pri['bpps_max'].to_list()).reshape(-1),color = "#C0C0C0")

In [ ]:
def read_bpps_max(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").max(axis=1))
    return bpps_arr

train['bpps_sum'] = read_bpps_max(train)
pri['bpps_sum'] = read_bpps_max(pri)
pub['bpps_sum'] = read_bpps_max(pub)

sns.distplot(np.array(train['bpps_max'].to_list()).reshape(-1),color = "#000000")
sns.distplot(np.array(pub['bpps_max'].to_list()).reshape(-1),color = "#0000FF")
sns.distplot(np.array(pri['bpps_max'].to_list()).reshape(-1),color = "#ADD8E6")

In [ ]:
def read_bpps_nb(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps = np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy")
        bpps_nb = (bpps > 0).sum(axis=0) / bpps.shape[0]
        bpps_arr.append(bpps_nb)
    return bpps_arr 

train['bpps_nb'] = read_bpps_nb(train)
pri['bpps_nb'] = read_bpps_nb(pri)
pub['bpps_nb'] = read_bpps_nb(pub)

sns.distplot(np.array(train['bpps_nb'].to_list()).reshape(-1),color="Blue")
sns.distplot(np.array(pub['bpps_nb'].to_list()).reshape(-1),color="Green")
sns.distplot(np.array(pri['bpps_nb'].to_list()).reshape(-1),color="Red")

In [ ]:
def mk_pair_map(structure, type = 'pm'):
    pm = np.full(len(structure), -1, dtype=int)
    pd = np.full(len(structure), -1, dtype=int)
    queue = []
    
    for i, s in enumerate(structure):
        if s == "(":
            queue.append(i)
        elif s == ")":
            j = queue.pop()
            pm[i] = j
            pm[j] = i
            pd[i] = i-j
            pd[j] = i-j
            
    if type == 'pm':
        return pm
    elif type == 'pd':
        return pd

In [ ]:
train['pair_map'] = train.structure.apply(mk_pair_map, type='pm')
pub['pair_map'] = pub.structure.apply(mk_pair_map, type='pm')
pri['pair_map'] = pri.structure.apply(mk_pair_map, type='pm')

train_list = np.array(train['pair_map'].to_list()).reshape(-1)
pub_list = np.array(pub['pair_map'].to_list()).reshape(-1)
pri_list = np.array(pri['pair_map'].to_list()).reshape(-1)

sns.distplot(train_list[~train_list<0],color="#800080")
sns.distplot(pub_list[~pub_list<0],color="#00FF00")
sns.distplot(pri_list[~pri_list<0],color="#FFFF00")

In [ ]:

train['pair_dist'] = train.structure.apply(mk_pair_map, type='pd')
pub['pair_dist'] = pub.structure.apply(mk_pair_map, type='pd')
pri['pair_dist'] = pri.structure.apply(mk_pair_map, type='pd')

train_list = np.array(train['pair_dist'].to_list()).reshape(-1)
pub_list = np.array(pub['pair_dist'].to_list()).reshape(-1)
pri_list = np.array(pri['pair_dist'].to_list()).reshape(-1)

sns.distplot(train_list[~train_list<0],color="#800000")
sns.distplot(pub_list[~pub_list<0],color="#008000")
sns.distplot(pri_list[~pri_list<0],color="#E55451")

In [ ]:
bpps_files = os.listdir('../input/stanford-covid-vaccine/bpps/')
plt.style.use('default')
fig, axs = plt.subplots(5, 5, figsize=(15, 15))
axs = axs.flatten()
for i, f in enumerate(bpps_files):
    if i == 25:
        break
    example_bpps = np.load(f'../input/stanford-covid-vaccine/bpps/{f}')
    axs[i].imshow(example_bpps)
    axs[i].set_title(f)
plt.tight_layout()
plt.show()

In [ ]:
def get_ngrams_counters(sequences, n=2):
    output = Counter()
    for sequence in sequences:
        output += Counter([sequence[i:i+n] for i in range(len(sequence)-1)])
        
    return output

In [ ]:
train_ngram_sequence = get_ngrams_counters(train.sequence)
test_ngram_sequence = get_ngrams_counters(test.sequence)

# Used to sort by frequency.
train_ngram_sequence = dict(train_ngram_sequence.most_common(10000))
test_ngram_sequence = dict(test_ngram_sequence.most_common(10000))

plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.title(f'Sequence character bigram counts (train) ({len(train_ngram_sequence)} unique bigrams)')
plt.bar(dict(train_ngram_sequence).keys(), dict(train_ngram_sequence).values())

plt.subplot(1, 2, 2)
plt.title(f'Sequence character bigram counts (test) ({len(test_ngram_sequence)} unique bigrams)')
plt.bar(dict(test_ngram_sequence).keys(), dict(test_ngram_sequence).values())

plt.show()

In [ ]:
train_ngram_sequence = get_ngrams_counters(train.sequence, 3)
test_ngram_sequence = get_ngrams_counters(test.sequence, 3)

# Used to sort by frequency.
train_ngram_sequence = dict(train_ngram_sequence.most_common(10000))
test_ngram_sequence = dict(test_ngram_sequence.most_common(10000))

plt.figure(figsize=(25, 10))
plt.title(f'Sequence character trigram (train) ({len(train_ngram_sequence)} unique trigrams)')
plt.bar(dict(train_ngram_sequence).keys(), dict(train_ngram_sequence).values())
plt.xticks(rotation=45)

plt.figure(figsize=(25, 10))
plt.title(f'Sequence character trigram (test) ({len(test_ngram_sequence)} unique trigrams)')
plt.bar(dict(test_ngram_sequence).keys(), dict(test_ngram_sequence).values())
plt.xticks(rotation=45)

plt.show()

In [ ]:
# This function performs some basic statististical analysis on the various labels.

def do_analysis(df, column_name):
    all_vals = [y for x in df[column_name] for y in x]
    print(f"Analysis across all samples for {column_name}")
    print(f'Mean: {np.mean(all_vals)}')
    print(f'Max: {np.max(all_vals)}')
    print(f'Min: {np.min(all_vals)}')
#     print(f'Mode: {mode(all_vals).mode[0]}')
    print(f'STD: {np.std(all_vals)}')
    print()
    
    plt.hist(all_vals)
    plt.title(f'Histogram for {column_name} across all samples')
    plt.show()
    
    print("Statistics aggregated per sample")
    fig, axes = plt.subplots(1, 4, figsize=(15, 5), squeeze=False)

    df[column_name].apply(
        lambda x: np.mean(x)).plot(
            kind='hist',
            bins=50, ax=axes[0,0],
            title=f'Mean dist {column_name}')

    df[column_name].apply(
        lambda x: np.max(x)).plot(
            kind='hist',
            bins=50, ax=axes[0,1],
            title=f'Max dist {column_name}')

    df[column_name].apply(
        lambda x: np.min(x)).plot(
            kind='hist',
            bins=50, ax=axes[0,2],
            title=f'Min dist {column_name}')
    df[column_name].apply(
        lambda x: np.std(x)).plot(
            kind='hist',
            bins=50, ax=axes[0,3],
            title=f'Std {column_name}')
    plt.show()

In [ ]:
do_analysis(train, 'reactivity')

In [ ]:
do_analysis(train, 'deg_Mg_pH10')

We can do same analysis for each feauture same way.

Hope You like my analysis of Data. By using this feautures we can apply models on top of it and get desired results